# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     |Juan Alvaro Morales Ramirez Valadez   |
| **Fecha**     18/09/2025 |   |
| **Expediente*745903* |   |

## Validación

### Hemos estado usando `train_test_split` en nuestros modelos anteriores.

### ¿Por qué?

Para hacer un train_test_split, es decir, para dividir el dataset en una parte para entrenar el modelo y otra para evaluarlo, con el objetivo de que el modelo pueda aprender realmente a partir de una porción de los datos y validar su desempeño en datos que no ha visto antes. Esto evita que el modelo se machetee (overfitting), es decir, que solo funcione bien con los datos de entrenamiento pero no generalice correctamente a datos nuevos o futuros, mejorando así su capacidad para hacer predicciones o estimaciones confiables en escenarios reales.

### Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

No, porque si usamos todos los datos para entrenar, el modelo no estaría aprendiendo realmente, sino más bien memorizando esos datos específicos. Esto puede generar un R2 muy acertado con ese conjunto de entrenamiento, pero cuando le presentemos datos nuevos, que no ha visto antes, el modelo no generalizará bien y tendrá un rendimiento pobre. Por eso es fundamental reservar una parte de los datos para validación o prueba, para evaluar si el modelo puede predecir correctamente fuera de la muestra que usó para entrenar.

# ------------------------------------------------------------------------------------------

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
df = pd.read_excel('Motor Trend Car Road Tests.xlsx')
df = df.drop(columns ='model', axis = 1)

x = df.drop(columns='mpg')
y = df['mpg']

mse=[]

for i in range(len(df)):
    #1-Sacar una muestra del dataset
    x_train = x.drop(i)
    y_train = y.drop(i)

    x_test = x.iloc[[i]]
    y_test = y.iloc[i]
   
    #2-Entrena tu modelo con las n-1 muestras
    modelo = LinearRegression()
    modelo.fit(x_train, y_train)

    #3- Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
    y_pred = modelo.predict(x_test)
    mse_i = mean_squared_error([y_test], [y_pred])
   
    #4- Regresa la muestra al dataset.
    mse.append(mse_i)
   
    #5 -Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento veces para muestras.
    #6- Calcula la media y desviación estándar de los métricos guardados.
    mse_mean = np.mean(mse)
   
    mse_std = np.std(mse)

print(round(mse_mean,3))
print(round(mse_std,3)) 

12.182
17.067


### Interpreta.

El MSE mean indica en promedio que tanto se alejan las predicciones del modelo sobre los datos reales.

El STD mean indcia cuanto vario mi error a lo largo de cada muestra.

En este caso la variabilidad es mas grande que mi error medio, lo que indica un bajo desmepeño del modelo, osea gran variabilidad en los errores de mis muestras

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [11]:
from sklearn.datasets import fetch_california_housing

df = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = df.data
y = df.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [79]:
import numpy as np
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Mezclar las filas aleatoriamente
X, y = shuffle(X, y)

# Definir el número de folds
k = 10
n_samples = X.shape[0]
fold_size = n_samples // k

mse_scores = []

# k-fold manual
for i in range(k):
    #Definir el tamaño de cada fold
    start = i * fold_size
    end = (i + 1) * fold_size if i != k - 1 else n_samples 

    # Seleccionar el test
    X_test = X[start:end]
    y_test = y[start:end]

    # Unir el resto de fold para hacer el entrenamiento
    X_train = np.concatenate((X[:start], X[end:]), axis=0)
    y_train = np.concatenate((y[:start], y[end:]), axis=0)

    # Modelo
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Evaluar modelo
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

# Resultados
mse_mean = np.mean(mse_scores)
mse_std = np.std(mse_scores)

print(round(mse_mean,3))
print(round(mse_std,3)) 

0.527
0.035


### Interpreta.

El MSE promedio de 0.527 indica que el modelo tiene un buen desempeño, con predicciones cercanas a los valores reales en promedio, mientras que la baja desviación estándar de 0.035 muestra que este desempeño es consistente y estable a lo largo de las diferentes particiones del dataset, lo que sugiere que el modelo generaliza bien y no presenta grandes variaciones en su error entre los distintos folds.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3